# Klasy

In [1]:
class Gadget:
    def __init__(self, id: int, name: str) -> None:
        self.id = id
        self.name = name

    def use(self) -> None:
        print(f"Using {self.name} with Id#{self.id}")

    def __repr__(self) -> str:
        return f"Gadget(id={self.id}, name={self.name!r})"

    def __str__(self) -> str:
        return f"Gadget - id: {self.id}; name: {self.name}"
    
    def __eq__(self, other):
        return (self.id, self.name) == (other.id, other.name)

In [2]:
(5).__add__(7)

12

In [3]:
g_1 = Gadget(1, "ipad")

In [4]:
type(g_1)

__main__.Gadget

In [5]:
id(g_1)

2128794721328

In [6]:
g_1.name = "ipad 2.0"

In [7]:
g_1.use()

Using ipad 2.0 with Id#1


In [8]:
print(g_1)

Gadget - id: 1; name: ipad 2.0


In [9]:
g_1

Gadget(id=1, name='ipad 2.0')

In [10]:
g_2 = eval(repr(g_1))

In [11]:
g_2

Gadget(id=1, name='ipad 2.0')

In [12]:
g_1 == g_2

True

In [13]:
g_1.__eq__(g_2)

True

# Metody `__new__` & `__init__`

In [16]:
class Dummy:
    def __new__(cls, *args):
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls)
        obj.extra_attribute = "EXTRA"
        print(f"Object {obj} has been created...")
        return obj

    def __init__(self, *args) -> None:
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: { self.__dict__}")

In [ ]:
d = Dummy(1, "one")

## Kiedy używać `__new__`?

In [ ]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")

        for i, item in enumerate(list):
            self[i] = item.upper()

In [ ]:
UppercaseTuple(['one', 'two', 'three'])

In [ ]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [ ]:
UppercaseTuple(['one', 'two', 'three'])

# Metody statyczne i metody klasy

In [ ]:
class CountedObject(object):
    count = 0   # statyczna składowa
    
    def __init__(self):
        CountedObject.count += 1
    
    @staticmethod  # statyczna metoda
    def get_count():
        return CountedObject.count

In [ ]:
lst = [CountedObject() for i in range(10)]

In [ ]:
CountedObject.count

10

In [ ]:
CountedObject.get_count()

10

In [ ]:
lst[3].count = 42

In [ ]:
class Person:
    name = "unknown"

In [ ]:
p1 = Person()

In [ ]:
p1.name

'unknown'

In [ ]:
p1.name = "Jan"

In [ ]:
p1.name

'Jan'

In [ ]:
class Date:
    year = 2023

    def __init__(self, day, month, year = None):
        self.day = day
        self.month = month
        if year:
            self.year = year 
    
    @classmethod
    def from_string(cls, date_as_string):        
        day, month, year = date_as_string.split('-')
        return cls(int(day), int(month), int(year)) # utworzenie instancji klasy cls

    @classmethod
    def update_default_year(cls, value):
        cls.year = value

In [ ]:
Date.year

2023

In [ ]:
d1 = Date(25, 9)

In [ ]:
d1.year

2023

In [ ]:
d1.__dict__

{'day': 25, 'month': 9}

In [ ]:
Date.update_default_year(2024)

In [ ]:
d1.year

2024

In [ ]:
d2 = Date.from_string("25-9-2023")

In [ ]:
d2.__dict__

{'day': 25, 'month': 9, 'year': 2023}

In [ ]:
d1.from_string("3-3-2022")

# Deskryptor

## Non-data descriptor

In [ ]:
import os

class DirectorySize:
    def __get__(self, instance, owner_class):
        print(f'Access to {instance} using descriptor {self}')
        return len(os.listdir(instance.directory_name))
    

class Directory:
    size = DirectorySize() # descriptor instance

    def __init__(self, directory_name):
        self.directory_name = directory_name # regular instance attribute

In [ ]:
local_dir = Directory('.')

In [ ]:
local_dir.__dict__

{'directory_name': '.'}

In [ ]:
local_dir.size

Access to <__main__.Directory object at 0x7f08f957dc10> using descriptor <__main__.DirectorySize object at 0x7f08f9535250>


1

## Data descriptor

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        logging.info('Setting names: %r and %r', self.public_name, self.private_name)

    def __get__(self, instance, owner_class=None):
        value = getattr(instance, self.private_name)
        logging.info('Accessing %r.%r giving %r', instance, self.public_name, value)
        return value

    def __set__(self, instance, value):
        logging.info('Updating %r.%r to %r', instance, self.public_name, value)
        setattr(instance, self.private_name, value)


class Person:
    age = LoggedAccess()             # Descriptor instance
    name = LoggedAccess()

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1                   # Calls both __get__() and __set__()

INFO:root:Setting names: 'age' and '_age'
INFO:root:Setting names: 'name' and '_name'


In [ ]:
p1 = Person("Jan", 22)

INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'name' to 'Jan'
INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'age' to 22


In [ ]:
p1.age

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'age' giving 22


22

In [ ]:
p1.age = 44

INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'age' to 44


In [ ]:
p1.age

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'age' giving 44


44

In [ ]:
p1.name

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'name' giving 'Jan'


'Jan'

In [ ]:
p1.birthday()

INFO:root:Accessing <__main__.Person object at 0x7f08f948cc50>.'age' giving 44
INFO:root:Updating <__main__.Person object at 0x7f08f948cc50>.'age' to 45


### ReadOnlyProperty

In [ ]:
class ReadOnlyProperty:
    def __init__(self, fget):
        self.fget = fget

    def __get__(self, instance, owner):
        return self.fget(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("Attribute is read-only")

In [ ]:
class Data:
    def __init__(self, data: int, name: str = "default") -> None:
        self._data = data
        self._name = name

    # def data(self) -> int:
    #    return self._data
    
    # data = ReadOnlyProperty(data)

    @ReadOnlyProperty
    def data(self):
        return self._data
    
    @property
    def name(self):
        return self._name

In [ ]:
data1 = Data(42)

In [ ]:
data1.data

42

In [ ]:
data1.data = 665

AttributeError: Attribute is read-only

# Slots

In [ ]:
class Pixel:
    __slots__ = ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y

In [ ]:
p1 = Pixel(20, 23)

In [ ]:
p1.__dict__

AttributeError: 'Pixel' object has no attribute '__dict__'

In [ ]:
p1.x

20

In [ ]:
p1.y

23

In [ ]:
class ColorPixel(Pixel):

    __slots__ = ('color')

    def __init__(self, x, y, color):
        super().__init__(x, y)
        self.color = color

In [ ]:
cp1 = ColorPixel(10, 20, 233)

In [ ]:
cp1.__dict__

AttributeError: 'ColorPixel' object has no attribute '__dict__'

In [ ]:
cp1.color

233